In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver

In [2]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [4]:
def chat_node(state: ChatState):
    #take user query from state
    messages = state['messages']
    #send to llm
    response = llm.invoke(messages).content
    #store response in state
    return {
        'messages': [AIMessage(content=response)]
    }

In [5]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)

#add nodes
graph.add_node("chat_node", chat_node)

graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

chatbot = graph.compile(checkpointer=checkpointer)

In [6]:
# chatbot

In [7]:
# initial_state = {
#     "messages": [HumanMessage(content="What is the capital of india")]
# }

# final_state = chatbot.invoke(initial_state)

In [8]:
# final_state

In [9]:
thread_id = '1'
while True:
    user_message = input("Type here: ")
    print("User: ", user_message)
    if user_message.strip().lower() in ['exit', 'quit', 'bye']:
        break
    config={"configurable":{"thread_id": thread_id}}
    response = chatbot.invoke({'messages': [HumanMessage(content=user_message)]}, config=config)
    
    print("AI: ", response["messages"][-1].content)

User:  my name is chintamani pala
AI:  It's nice to meet you, Chintamani Pala.
User:  what is my name
AI:  Your name is Chintamani Pala.
User:  sum of 1 to 10
AI:  The sum of numbers from 1 to 10 is 55.
User:  numtiply with 2
AI:  55 multiplied by 2 is 110.
User:  exit


In [10]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='my name is chintamani pala', additional_kwargs={}, response_metadata={}, id='04acf214-d846-4928-9dcf-2d7d3662c9d5'), AIMessage(content="It's nice to meet you, Chintamani Pala.", additional_kwargs={}, response_metadata={}, id='1bc25d2b-9c28-4e7d-b322-702408a366a3'), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}, id='a2322cd4-b402-42c0-a582-6ad259c5053a'), AIMessage(content='Your name is Chintamani Pala.', additional_kwargs={}, response_metadata={}, id='6df83cdd-ffcf-4c18-a7d8-30fa1b231be4'), HumanMessage(content='sum of 1 to 10', additional_kwargs={}, response_metadata={}, id='9f04a4c0-a63e-47e6-a3af-3753cd2d1f6a'), AIMessage(content='The sum of numbers from 1 to 10 is 55.', additional_kwargs={}, response_metadata={}, id='6ec8d379-4594-4a85-a918-c1f61e22bbc9'), HumanMessage(content='numtiply with 2', additional_kwargs={}, response_metadata={}, id='a2c58200-1cda-4b7b-9f60-2aa9e27de2cb'), AIMessa